# Original feature vs ours
This notebook compares the features from the TSV file, which made with the caffe model, and those from the extractor.   
We compare some images in the download directory.
The tsv file was made in advance by extracting these files.

In [1]:
import torch
import numpy as np
import PIL.Image

from eval_vl_glue import VoltaImageFeature, load_tsv
from eval_vl_glue.extractor import BUTDDetector

In [2]:
image_paths = [
    '../download/filled_with_0.png', 
    '../download/filled_with_255.png',
    '../download/000542.jpg', 
]

## Load the TSV file

In [3]:
tsv_path = '../vl_models/test_obj36.tsv'
feature_dict = load_tsv(tsv_path)
# Change raw features into inputs for transformers_volta
tsv_features = {k: VoltaImageFeature.from_dict(v) for k, v in feature_dict.items()}

## Configure extractor

In [4]:
model_path = '../download/resnet101_faster_rcnn_final_iter_320000.pt'
device = torch.device('cpu')
extractor = BUTDDetector()
extractor.load_state_dict(torch.load(model_path))
extractor = extractor.to(device).eval()

## Feature matching function
- Two tensors have 36 features each (excluding global image feature).     
  We match those features trying to minimize the sum of pair distances (cossim, absolute value of differences).

In [5]:
# a function to match nearest features in a greedy way
def greedy_match(v1, v2, func=None):
    func = func or (lambda x, y: (x - y).abs().mean(axis=-1))
    ds = func(v1[None], v2[:, None])
    n = ds.shape[0]
    not_available = ds.max().item() + 100

    pairs = []
    dists = torch.zeros((n,), dtype=torch.float32)
    for _ in range(n):
        dists[_] = ds.min().item()
        i = ds.argmin().item()
        c = int(i % n)
        r = int(i // n)
        pairs.append((r, c))
        ds[r] = not_available
        ds[:, c] = not_available
    return pairs, dists

In [6]:
def compare(v1, v2):
    # Element-wise absolute values
    pairs, dists = greedy_match(v1, v2, func=lambda x, y: (x - y).abs().mean(axis=-1))
    print('absolute mean:', torch.cat([v1, v2], axis=0).abs().mean().item())
    print('averaged absolute difference:', dists.mean().item())
    print('difference', dists)
    
    # Vector-wise cosine similarity
    pairs, dists = greedy_match(v1, v2, func=lambda x, y: -torch.nn.functional.cosine_similarity(x, y, dim=-1))
    dists *= -1
    print('averaged cossim:', dists.mean().item())
    print('cossim', dists)

## comparison

In [7]:
for image_path in image_paths:
    tsv_feature = tsv_features[image_path.split('/')[-1].split('.')[0]]
    
    # extract features from a image using the extractor
    image = PIL.Image.open(image_path)
    regions = extractor.detect(image)
    ext_feature = VoltaImageFeature.from_regions(regions)
    
    print(image_path)
    # We do not consider global image features.
    compare(tsv_feature.features[1:], ext_feature.features[1:])
    print()

/Users/taichi/research/eval_vl_glue/venv/lib/python3.9/site-packages/torch/nn/functional.py:3657: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
/Users/taichi/research/eval_vl_glue/venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


../download/filled_with_0.png
absolute mean: 0.5030784606933594
averaged absolute difference: 0.11184278875589371
difference tensor([0.0047, 0.0065, 0.0106, 0.0149, 0.0180, 0.0294, 0.0369, 0.0379, 0.0384,
        0.0423, 0.0467, 0.0493, 0.0594, 0.0606, 0.0637, 0.0645, 0.0659, 0.0753,
        0.0763, 0.0852, 0.0958, 0.0964, 0.0991, 0.0996, 0.1179, 0.1261, 0.1299,
        0.1310, 0.1444, 0.1654, 0.2163, 0.2515, 0.2841, 0.3150, 0.4064, 0.4610])
averaged cossim: 0.9530308246612549
cossim tensor([1.0000, 0.9999, 0.9998, 0.9996, 0.9996, 0.9986, 0.9982, 0.9979, 0.9972,
        0.9967, 0.9963, 0.9956, 0.9953, 0.9953, 0.9928, 0.9927, 0.9922, 0.9916,
        0.9909, 0.9888, 0.9872, 0.9825, 0.9807, 0.9801, 0.9783, 0.9766, 0.9729,
        0.9721, 0.9537, 0.9377, 0.9047, 0.8851, 0.8584, 0.7946, 0.6990, 0.5267])

../download/filled_with_255.png
absolute mean: 0.4254292845726013
averaged absolute difference: 0.12754279375076294
difference tensor([1.9365e-05, 4.1474e-03, 7.3723e-03, 7.9933e-03, 1.0046

- The averages of element-wise absolute difference ranges 0.1-0.16. These values are approximetly 20% of absolute mean of elements.
- The average of vector-wise cosine similarity is approximetly 0.95.
- Although two models are similar to some extent, these are not identical. 
- According to our debug, raw outputs of neural networks, such as rpn_cls_prob_reshape in the forward function, are slightly different and this seems to affect boundary candidates in the remove_small_boxes function.